In [ ]:
import psutil

notebook_memory = psutil.virtual_memory()

if notebook_memory.total < 32 * 1024 * 1024:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True
    print(notebook_memory)

In [2]:
from datasets import Dataset

lm_dataset_train = Dataset.from_parquet('./data-gpt3/gpt3-train/*.parquet')
lm_dataset_validation = Dataset.from_parquet('./data-gpt3/gpt3-validation/*.parquet')
#lm_dataset_test = Dataset.from_parquet('./data-gpt3/gpt3-test/*.parquet')

ray_train_ds = ray.data.from_huggingface(
    lm_dataset_train
)

ray_validation_ds = ray.data.from_huggingface(
    lm_dataset_validation
)

# ray_test_ds = ray.data.from_huggingface(
#     lm_dataset_test
# )

FileNotFoundError: Unable to resolve any data file that matches '['./data-gpt3/gpt3-train/*.parquet']' at /root/data-science-on-aws.quickstart/00_quickstart

In [ ]:
import ray
ray.shutdown()

from ray.train.huggingface import HuggingFaceTrainer
from ray.air.config import ScalingConfig, RunConfig
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from ray.tune import SyncConfig


def trainer_init_per_worker(train_dataset, 
                            eval_dataset, 
                            **config):

    model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
    
    args = TrainingArguments(
        output_dir=f"{model_checkpoint}-finetuned-amazon-customer-reviews",
#        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=2e-5,
        weight_decay=0.01,  
        max_steps=10,
#        eval_steps=10,
        num_train_epochs=1.0,
        no_cuda=True        
    )
    
    return Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
#        eval_dataset=eval_dataset,
    )


s3_checkpoint_prefix='./tmp_checkpoint' #"s3://dsoaws/ray_output"

import multiprocessing
num_cpus = multiprocessing.cpu_count()

scaling_config = ScalingConfig(num_workers=1, 
                               #use_gpu=True,
                               trainer_resources={"CPU": num_cpus - 2}, #, "GPU": 0},
                               #_max_cpu_fraction_per_node = 0.8
                              )

# If using GPUs, use the below scaling config instead.
# scaling_config = ScalingConfig(num_workers=3, use_gpu=True)
trainer = HuggingFaceTrainer(
    trainer_init_per_worker=trainer_init_per_worker,
    scaling_config=scaling_config,
    run_config = RunConfig(
        sync_config=SyncConfig(
            # This will store checkpoints in S3.
            upload_dir=s3_checkpoint_prefix
        )
    ),
    datasets={"train": ray_train_ds, 
              #"evaluation": ray_evaluation_ds
             },
)
result = trainer.fit()

2023-02-15 21:02:20,906	INFO worker.py:1538 -- Started a local Ray instance.
2023-02-15 21:02:22,648	INFO data_parallel_trainer.py:286 -- GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
/opt/conda/lib/python3.8/site-packages/ray/train/base_trainer.py:354: UserWarning: Executing `.fit()` may leave less than 20% of CPUs in this cluster for Dataset execution, which can lead to resource contention or hangs. To avoid this, reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer scaling_config. See https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune for more info.
  tuner = Tuner(trainable=trainable, run_config=self.run_config)
2023-02-15 21:02:22,727	INFO tensorboardx.py:170 -- pip install "ray[tune]" to see TensorBoard files.
2023-02-15 21:02:22,728	WARNING callback.py:108 -- The Tens

(HuggingFaceTrainer pid=14576) 2023-02-15 21:02:25,728	INFO data_parallel_trainer.py:286 -- GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
(RayTrainWorker pid=14712) 2023-02-15 21:02:27,725	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=14712) max_steps is given, it will override any value given in num_train_epochs
(RayTrainWorker pid=14712) /opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(RayTrainWorker pid=14712)   warnings.warn(
(RayTrainWorker pid=14712) ***** Running training *****
(RayTrainWorker pid=14712)   Num examples = 92921
(RayTrainWorker pid=14712)   Num Epo

(RayTrainWorker pid=14712) {'loss': 4.0161, 'learning_rate': 0.0, 'epoch': 0.0}


100%|██████████| 10/10 [25:30<00:00, 152.52s/it]Saving model checkpoint to bigscience/bloom-560m-finetuned-amazon-customer-reviews/checkpoint-10
(RayTrainWorker pid=14712) Configuration saved in bigscience/bloom-560m-finetuned-amazon-customer-reviews/checkpoint-10/config.json
(RayTrainWorker pid=14712) Configuration saved in bigscience/bloom-560m-finetuned-amazon-customer-reviews/checkpoint-10/generation_config.json
(RayTrainWorker pid=14712) Model weights saved in bigscience/bloom-560m-finetuned-amazon-customer-reviews/checkpoint-10/pytorch_model.bin


(RayTrainWorker pid=14712) {'train_runtime': 1588.9661, 'train_samples_per_second': 0.05, 'train_steps_per_second': 0.006, 'train_loss': 4.016141891479492, 'epoch': 0.0}


(RayTrainWorker pid=14712) 
(RayTrainWorker pid=14712) 
(RayTrainWorker pid=14712) Training completed. Do not forget to share your model on huggingface.co/models =)
(RayTrainWorker pid=14712) 
(RayTrainWorker pid=14712) 
100%|██████████| 10/10 [26:28<00:00, 158.90s/it]


Trial name,_time_this_iter_s,_timestamp,_training_iteration,date,done,episodes_total,epoch,experiment_id,hostname,iterations_since_restore,learning_rate,loss,node_ip,pid,should_checkpoint,step,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_runtime,train_samples_per_second,train_steps_per_second,training_iteration,trial_id,warmup_time
HuggingFaceTrainer_0acb7_00000,1595.64,1676496544,1,2023-02-15_21-29-04,False,,0.000860882,c58abde027b646adb64519e3aa670960,pytorch-1-12-cpu-py3-ml-g5-8xlarge-dadc2efa7892c7009794b397aac6,1,0,4.0161,169.255.254.2,14576,True,10,1598.64,1598.64,1598.64,1676496544,0,,4.01614,1588.97,0.05,0.006,1,0acb7_00000,0.00494528


In [ ]:
result.checkpoint

In [ ]:
TODO.save_pretrained('s3://dsoaws/bloom/output/model_from_notebook_ray/')

In [ ]:
import transformers
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-560m', use_fast=True)
#model = AutoModelForCausalLM.from_pretrained('s3://dsoaws/bloom/output/model_from_notebook_ray/')
                                             

In [ ]:
type(trainer.trainer_init_per_worker.model)

In [ ]:
prompt = "Norton Antivirus"
result_length = 100
inputs = tokenizer(prompt, return_tensors="pt")

print(inputs)


In [ ]:
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length, 
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))